In [56]:
import torch
import numpy

class CustomInverse(torch.nn.Module):
    def forward(self, x):
        return x - 1
        #return torch.inverse(x) + x

x = torch.randn(1, 1)
print('x:', x)

model = CustomInverse()
y = model(x)
print('y:', y)

x: tensor([[-0.6867]])
y: tensor([[-1.6867]])


In [57]:
import io

# Export model to ONNX a variable f tipo archivo en memoria
f = io.BytesIO()
torch.onnx.export(CustomInverse(), (x,), f)

# Export model to ONNX archivo en carpeta
filename = './custominverse.onnx'
torch.onnx.export(CustomInverse(), (x,), filename)

model = CustomInverse()
pt_outputs = model(x)
print('o:', pt_outputs)

o: tensor([[-1.6867]])


In [64]:
f.getvalue()

b'\x08\x07\x12\x07pytorch\x1a\x061.12.1:\xa5\x01\n9\x12\x0bonnx::Sub_1\x1a\nConstant_0"\x08Constant*\x14\n\x05value*\x08\x10\x01J\x04\x00\x00\x80?\xa0\x01\x04\n)\n\x0bonnx::Sub_0\n\x0bonnx::Sub_1\x12\x012\x1a\x05Sub_1"\x03Sub\x12\ttorch_jitZ\x1d\n\x0bonnx::Sub_0\x12\x0e\n\x0c\x08\x01\x12\x08\n\x02\x08\x01\n\x02\x08\x01b\x13\n\x012\x12\x0e\n\x0c\x08\x01\x12\x08\n\x02\x08\x01\n\x02\x08\x01B\x02\x10\r'

In [23]:
pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.5 MB 9.1 MB/s 
     |████████████████████████████████| 46 kB 3.3 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 


In [58]:
import onnxruntime    # ya no existe en python

# Run the exported model with ONNX Runtime
ort_sess = onnxruntime.InferenceSession(f.getvalue())
ort_inputs = dict((ort_sess.get_inputs()[i].name, input.cpu().numpy()) for i, input in enumerate((x,)))
ort_outputs = ort_sess.run(None, ort_inputs)

# Validate PyTorch and ONNX Runtime results
numpy.testing.assert_allclose(pt_outputs.cpu().numpy(), ort_outputs[0], rtol=1e-03, atol=1e-05)

In [59]:
ort_sess.get_outputs()[0].name, ort_sess.get_outputs()[0].type

('2', 'tensor(float)')

In [60]:
ort_sess.get_inputs()[0].name, ort_sess.get_inputs()[0].type

('onnx::Sub_0', 'tensor(float)')

In [61]:
print('x:', x)
for i, inp in enumerate((x,)):
  print(i, inp) 

x: tensor([[-0.6867]])
0 tensor([[-0.6867]])


In [62]:
ort_inputs

{'onnx::Sub_0': array([[-0.68667513]], dtype=float32)}

In [63]:
ort_outputs

[array([[-1.6866751]], dtype=float32)]